In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

In [ ]:
import os
if os.path.exists('cases.sql'):
    os.remove('cases.sql')

# System Recording

If you need to focus on a smaller part of your model, it may be useful to attach a case recorder to a particular `System`. There are slightly different options when recording from these objects. System recording can only be used in serial running environments and cannot currently be used in parallel.

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.core.system.System", options_dict="recording_options")

```{note}
Note that the `excludes` option takes precedence over the `includes` option.
```

## System Recording Example

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src92", get_code("openmdao.test_suite.components.sellar_feature.SellarDerivatives"), display=False)

:::{Admonition} `SellarDerivatives` class definition 
:class: dropdown

{glue:}`code_src92`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.components.sellar_feature import SellarDerivatives

prob = om.Problem(model=SellarDerivatives())
prob.setup()

recorder = om.SqliteRecorder("cases.sql")

obj_cmp = prob.model.obj_cmp
obj_cmp.add_recorder(recorder)
obj_cmp.recording_options['includes'] = ['*']
obj_cmp.recording_options['excludes'] = ['obj_cmp.x']

prob.model.nonlinear_solver = om.NonlinearBlockGS()
prob.model.nonlinear_solver.options['use_apply_nonlinear'] = True

prob.run_model()

In [ ]:
prob.cleanup()

cr = om.CaseReader("cases.sql")

system_cases = cr.list_cases('root.obj_cmp')

In [ ]:
assert len(system_cases) == 7

In [ ]:
case = cr.get_case(system_cases[0])

case.inputs

In [ ]:
assert sorted(case.inputs.keys()) == ['y1', 'y2', 'z']